<a href="https://colab.research.google.com/github/ematala/satellite-segmentation/blob/main/SlidingWindowPadding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sys
import numpy
import matplotlib.pyplot as plt

In [7]:
# Class Names
classes = numpy.array(['No Data', 'Cultivated Land', 'Forest', 'Grassland', 'Shrubland',
                    'Water', 'Wetlands', 'Tundra', 'Artificial Surface',
                    'Bareland', 'Snow and Ice'])

In [8]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# load training data from npz file
# data = numpy.load('/content/gdrive/MyDrive/train.npz')

In [9]:
# load test data from npz file
public_test_data = numpy.load('/content/gdrive/MyDrive/public_test.npz')
test_bands = public_test_data['bands']

## Exploratory

In [6]:
n_months = 12
n_classes = len(classes)
n_channels = 6

In [ ]:
# Check for RGB Values in range
for month in range(n_months):
  rgb_img = test_bands[month][:, :, (2,1,0)]
  mean = rgb_img.mean()
  print(rgb_img.max(),"max")
  print(rgb_img.min(),"min")

In [ ]:
# plot the train set
f, axes = plt.subplots(1, n_months, figsize=(200,200))

for month in range(n_months):
    axes[month].imshow(test_bands[month][:, :, 0:3])

In [ ]:
# plot the test set
f, axes = plt.subplots(1, n_months, figsize=(300,300))

for month in range(n_months):
    axes[month].imshow(padded_bands[month][:, :, 0:3])

# Temporary CNN for Testing

In [6]:
# Tom CNN 

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils.np_utils import to_categorical

model = Sequential()
model.add(Conv3D(32, kernel_size=(1, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(12,33,33,6)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Conv3D(64, kernel_size=(1, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(11, activation='softmax'))

# Compile the model
model.compile(
    'Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 12, 31, 31, 32)    1760      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 6, 15, 15, 32)     0         
_________________________________________________________________
batch_normalization (BatchNo (None, 6, 15, 15, 32)     128       
_________________________________________________________________
dropout (Dropout)            (None, 6, 15, 15, 32)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 6, 13, 13, 64)     18496     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 6, 6, 64)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 6, 6, 64)       2

In [7]:
from sklearn.model_selection import train_test_split

# separate bands and labels:
train_data = data['bands']
labels = data['lulc']

# remove no_data
labels_no_data = labels[labels != 0]
train_data_no_data = train_data[labels != 0, :, :, :, :]

train = {}
validation = {}
# with no_data removed
train['features'], validation['features'], train['labels'], validation['labels'] = train_test_split(train_data_no_data, labels_no_data, test_size=0.2, random_state=0)

val = validation['labels']
train['labels'] = to_categorical(train['labels'], 11)
validation['labels'] = to_categorical(validation['labels'], 11)

In [15]:
# Fit model
history = model.fit(train['features'], train['labels'], batch_size=64, epochs=50, validation_data=(validation['features'], validation['labels']))

Epoch 1/50
109/109 [==============================] - 6s 45ms/step - loss: 0.5075 - accuracy: 0.8251 - val_loss: 1.5985 - val_accuracy: 0.6947
Epoch 2/50
109/109 [==============================] - 4s 40ms/step - loss: 0.4324 - accuracy: 0.8463 - val_loss: 1.3234 - val_accuracy: 0.7309
Epoch 3/50
109/109 [==============================] - 4s 40ms/step - loss: 0.4014 - accuracy: 0.8626 - val_loss: 1.3994 - val_accuracy: 0.7148
Epoch 4/50
109/109 [==============================] - 4s 40ms/step - loss: 0.3586 - accuracy: 0.8783 - val_loss: 1.4225 - val_accuracy: 0.6947
Epoch 5/50
109/109 [==============================] - 4s 40ms/step - loss: 0.3296 - accuracy: 0.8826 - val_loss: 1.2893 - val_accuracy: 0.7309
Epoch 6/50
109/109 [==============================] - 4s 40ms/step - loss: 0.3136 - accuracy: 0.8894 - val_loss: 1.8652 - val_accuracy: 0.7355
Epoch 7/50
109/109 [==============================] - 4s 40ms/step - loss: 0.2859 - accuracy: 0.9028 - val_loss: 1.6661 - val_accuracy: 0.7263

In [16]:
model.save('/content/gdrive/MyDrive/DummyModel')

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/DummyModel/assets


# Padded Sliding Window

In [11]:
import keras
from keras.models import load_model

model = keras.models.load_model("/content/gdrive/MyDrive/DummyModel")

In [12]:
padded_bands = []
for month in range(12):
    padded_bands.append(numpy.pad(test_bands[month], ((16,16), (16,16), (0,0)), "symmetric"))

padded_bands = numpy.asarray(padded_bands)

In [13]:
patches = []
predictions = []

for h in range(500):
  for w in range(500):
    patches.append(padded_bands[:, w:w+33, h:h+33, :])
  temp_preds = model.predict(numpy.asarray(patches))
  for pred in temp_preds:
      predictions.append(numpy.argmax(pred))
  del patches
  del temp_preds
  patches = []
predictions = numpy.asarray(predictions)
predictions = predictions.reshape(500, 500)


[[1 1 1 ... 1 2 2]
 [1 1 1 ... 1 2 2]
 [1 1 1 ... 2 2 2]
 ...
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]]
